## 전처리된 전체 데이터 하나로 통합

### 데이터 통합 (Data Merge)

이 노트북은 서울시 미세먼지 데이터, 기상 데이터, 어린이집 데이터를 분석에 적합한 형태로 통합하기 위해 작성되었습니다.

- 데이터 로드 및 기본 정보 확인
- 컬럼 및 데이터 수정
- 데이터 통합

데이터를 통합하여 모델링에 필요한 깨끗하고 일관된 형태의 데이터셋을 구축하는 데 목적이 있습니다.

### 1. 데이터 로드 및 기본 설정

In [1]:
#  Jupyter Notebook에서 외부 모듈(.py 파일)을 수정할 경우,
#  매번 커널을 재시작하거나 수동으로 reload하지 않아도
#  자동으로 가장 최신 상태로 import 되도록 설정합니다.

# %load_ext autoreload:
# IPython의 autoreload 확장 기능을 불러옵니다.
# 이 기능은 외부 .py 파일이 변경될 때 자동으로 다시 import 하도록 도와줍니다.

# %autoreload 2:
# 모든 모듈을 cell 실행 시마다 자동으로 reload합니다.
# (즉, 외부 .py 파일을 수정하고 저장만 해도 바로 반영됨)

# 사용 예:
#   - utils.py, visualization.py 등 자주 수정하는 모듈이 있는 경우
#   - 매번 커널 재시작 없이 편리하게 함수 변경 사항을 반영하고 싶을 때

# 단, 모듈의 내부 상태를 유지하고 싶은 경우에는 주의해서 사용해야 합니다.

%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# 프로젝트 루트 경로를 sys.path에 추가
project_root = os.path.abspath("..")  # notebooks 폴더 기준 상위 폴더
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
import pandas as pd
from scripts.utils import setup_font

# 한글 폰트 설정
setup_font()

Current OS: Darwin
Setting macOS font: AppleGothic
Current font settings: ['AppleGothic']


In [4]:
# 원본 데이터가 저장된 폴더 경로 설정
folder_path = '../data/processed'
save_dir = '../data/processed/result'

# 데이터 파일명 설정
air_file = 'air_quality_merged.csv'
weather_file = 'daily_weather_preprocessed.csv'
daycare_file = 'daycarecenter_preprocessed.csv'

# 파일 읽기
air_df = pd.read_csv(os.path.join(folder_path, air_file))
weather_df = pd.read_csv(os.path.join(folder_path, 'weather', weather_file))
daycare_df = pd.read_csv(os.path.join(folder_path, 'daycarecenter', daycare_file))

# 저장 폴더 생성
os.makedirs(save_dir, exist_ok=True)


### 2. 데이터 형식 확인 및 컬럼명 변경
- 데이터 형식 확인
- 통합을 위한 컬럼명 통일
- 전체 데이터 통합
- 컬럼 순서 변경

In [5]:
# 미세먼지 데이터
air_df.head()

,date,pm25,pm10,측정소명
0,2018-01-01,75.0,28.0,강남구
1,2018-01-02,72.0,25.0,강남구
2,2018-01-03,67.0,31.0,강남구
3,2018-01-04,79.0,42.0,강남구
4,2018-01-05,105.0,32.0,강남구


In [6]:
# 기상 데이터
weather_df.head()

,지점명,날짜,평균기온(°C),일강수량(mm),평균 풍속(m/s)
0,강남,2018-01-01,0.3,0.0,1.4
1,강남,2018-01-02,-0.9,0.0,1.7
2,강남,2018-01-03,-3.7,0.0,2.0
3,강남,2018-01-04,-3.7,0.0,1.1
4,강남,2018-01-05,-1.5,0.0,1.5


In [7]:
# 어린이집 데이터
daycare_df.head()

,시군구,어린이집명,주소,위도,경도,측정소,측정소까지거리(km)
0,구로구,명화어린이집,서울특별시 구로구 개봉로17다길 6 (개봉동),37.493450,126.855103,구로구,3.134825
1,구로구,꿈이있는어린이집,서울특별시 구로구 벚꽃로 484 화원사회종합복지관내 (구로동),37.497329,126.880142,구로구,0.885956
2,구로구,이편한어린이집,서울특별시 구로구 신도림로 16 대림아파트 관리동1층 (신도림동),37.505801,126.880712,구로구,1.179900
3,구로구,별나라어린이집,"서울특별시 구로구 신도림로11가길 36 5동 101호(신도림동, 미성아파트)",37.508274,126.876108,구로구,1.663649
4,구로구,미래어린이집,서울특별시 구로구 구로동로20길 57 (구로동),37.488049,126.887631,구로구,1.155982


In [8]:
# 미세먼지 데이터
# 컬럼명 변경
air_df = air_df.rename(columns={'date': '날짜'})

In [9]:
# 기상 데이터
# 컬럼명 변경
weather_df = weather_df.rename(columns={'지점명': '측정소명'})
# 지역 이름 정제
location_mapping = {
    '강남': '강남구',
    '강동': '강동구',
    '강북': '강북구',
    '강서': '강서구',
    '관악': '관악구',
    '광진': '광진구',
    '구로': '구로구',
    '금천': '금천구',
    '노원': '노원구',
    '도봉': '도봉구',
    '동대문': '동대문구',
    '동작': '동작구',
    '마포': '마포구',
    '서대문': '서대문구',
    '서초': '서초구',
    '성동': '성동구',
    '성북': '성북구',
    '송파': '송파구',
    '양천': '양천구',
    '영등포': '영등포구',
    '용산': '용산구',
    '은평': '은평구',
    '종로': '종로구',
    '중구': '중구',
    '중랑': '중랑구'
}
weather_df['측정소명'] = weather_df['측정소명'].map(location_mapping)


In [10]:
# 기상데이터 + 미세먼지 데이터 통합
merged_df = pd.merge(air_df, weather_df, 
                    on=['날짜', '측정소명'],  # 기준이 되는 컬럼
                    how='inner')  # inner join 사용

merged_df.head()

,날짜,pm25,pm10,측정소명,평균기온(°C),일강수량(mm),평균 풍속(m/s)
0,2018-01-01,75.0,28.0,강남구,0.3,0.0,1.4
1,2018-01-02,72.0,25.0,강남구,-0.9,0.0,1.7
2,2018-01-03,67.0,31.0,강남구,-3.7,0.0,2.0
3,2018-01-04,79.0,42.0,강남구,-3.7,0.0,1.1
4,2018-01-05,105.0,32.0,강남구,-1.5,0.0,1.5


In [11]:
# 어린이집 데이터
# 컬럼명 변경
daycare_df = daycare_df.rename(columns={'측정소': '측정소명', '시군구': '어린이집 위치'})
# 불필요한 컬럼 제거
columns_to_drop = ['주소']
daycare_df = daycare_df.drop(columns=columns_to_drop)
daycare_df = daycare_df.rename(columns={'시군구': '어린이집 위치' ,'date': '날짜'})


In [12]:
# 기상데이터 + 미세먼지 데이터 + 어린이집 데이터 통합
daycare_air_quality_with_distance = pd.merge(merged_df, daycare_df, 
                    on=['측정소명'],  # 기준이 되는 컬럼
                    how='inner')  # inner join 사용

daycare_air_quality_with_distance.head()

,날짜,pm25,pm10,측정소명,평균기온(°C),일강수량(mm),평균 풍속(m/s),어린이집 위치,어린이집명,위도,경도,측정소까지거리(km)
0,2018-01-01,75.0,28.0,강남구,0.3,0.0,1.4,강남구,성아어린이집,37.484064,127.043581,3.745348
1,2018-01-01,75.0,28.0,강남구,0.3,0.0,1.4,강남구,선재어린이집,37.501472,127.056348,1.936213
2,2018-01-01,75.0,28.0,강남구,0.3,0.0,1.4,강남구,바롬어린이집,37.525327,127.036350,1.339645
3,2018-01-01,75.0,28.0,강남구,0.3,0.0,1.4,강남구,연화어린이집,37.517259,127.037184,0.951706
4,2018-01-01,75.0,28.0,강남구,0.3,0.0,1.4,강남구,논현어린이집,37.511002,127.026869,1.998995


In [13]:
# 컬럼 순서 변경으로 가독성 향상
cols = ['날짜', '측정소명', 'pm10', 'pm25', '평균기온(°C)', '일강수량(mm)', '평균 풍속(m/s)', '어린이집 위치','위도','경도', '어린이집명', '측정소까지거리(km)']
daycare_air_quality_with_distance = daycare_air_quality_with_distance[cols]

daycare_air_quality_with_distance.head()

,날짜,측정소명,pm10,pm25,평균기온(°C),일강수량(mm),평균 풍속(m/s),어린이집 위치,위도,경도,어린이집명,측정소까지거리(km)
0,2018-01-01,강남구,28.0,75.0,0.3,0.0,1.4,강남구,37.484064,127.043581,성아어린이집,3.745348
1,2018-01-01,강남구,28.0,75.0,0.3,0.0,1.4,강남구,37.501472,127.056348,선재어린이집,1.936213
2,2018-01-01,강남구,28.0,75.0,0.3,0.0,1.4,강남구,37.525327,127.036350,바롬어린이집,1.339645
3,2018-01-01,강남구,28.0,75.0,0.3,0.0,1.4,강남구,37.517259,127.037184,연화어린이집,0.951706
4,2018-01-01,강남구,28.0,75.0,0.3,0.0,1.4,강남구,37.511002,127.026869,논현어린이집,1.998995


### 3. 최종 데이터 파일 저장
- 학습을 위한 최종 데이터 파일 저장

In [14]:
from scripts.utils import save_to_csv

save_to_csv(daycare_air_quality_with_distance, output_dir=save_dir, file_name='daycare_air_quality_with_distance')